In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import seaborn as sns
sns.set()

In [10]:
# function to calculate my measure of stock volatility
# daily range divided by the median daily value

def wills_volatility(ticker):
    x = yf.Ticker(ticker).history(period='MAX')
    ans = []
    high = x['High']
    low = x['Low']
    mean = (abs(high)+abs(low))/2
    dr = abs(high-low)
    ans = (((dr/mean)*1000)+10)
    hi = pd.DataFrame(data=x)
    hi['Wills Volatility Measure'] = ans
    hi['Date'] = hi.index
    return(hi)

In [11]:
test = wills_volatility("^FTSE")
# This function works!

In [12]:
# I will get the past 5 years of data to test suitability of this measure

vix = yf.Ticker("^VIX").history(period='5Y')
SandP500 = yf.Ticker("^GSPC").history(period='5Y')

In [13]:
# This code calculates the daily range of a stock
# I then divide it by the median daily value to standardise
High = SandP500['High'] 
Low = SandP500['Low']
day_range = abs((High)-(Low))
day_mean = (abs(High)+abs(Low))/2
SDR = ((day_range/(day_mean))*1000)+10

In [14]:
# test correlation between standardised daily range I calculated
# and the vix index to check suitability.
SDR.corr(vix['Close'])

0.8068185350337577

In [15]:
# 0.807 shows a strong positive correlation
# therefore I will use this as a measure of volatility

In [ ]:
# Now I will create a function that performs this calculation
# automatically for any ticker from yahoo finance
# and returns the augmented Pandas dataframe

In [ ]:
# I will will export to a csv file and then read back in
# to use within the alignment alogrithm to align dates
# and calculate moving averages of the volatility metric
df_test.to_csv('Algo Test.csv')
ftse = pd.read_csv('Algo Test.csv')

In [ ]:
def Align_my_data(data):
    hurricanes = pd.read_csv('hurricanes_scraped.csv') # read in hurricane data
    hurricanes = hurricanes.rename(columns={'Date of closest approach': 'Date'})
    for x in range(0,len(hurricanes)): # this loop formats date column to match dates
        hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")
    zing = data
    MA21 = [] # I will now create 21 point (day) moving average series
    for y in range(0,21):
        MA21.append('NaN')
    for x in range(21,len(zing)):
        MA21.append((sum(zing['Wills Volatility Measure'][x-21:x])/21))
    zing['MA21'] = MA21
    Trailing21 = [] # I will also create a 21 point (day) trailing moving average series
    for y in range(0,35):
        Trailing21.append('NaN')
    for x in range(35,len(zing)):
        Trailing21.append((sum(zing['Wills Volatility Measure'][x-35:x-14])/21))
    zing['Trailing MA'] = Trailing21
    zing_box = []
    date_box = []
    hurricane_box = []
    MA21_box = []
    Trailing21_box = []
    for x in range(0,len(hurricanes['Date'])): # loops over hurricane hit dates
        for y in range(0, len(zing['Date'])): # loops over dates of time series to align with hurricanes
            if hurricanes['Date'][x] == zing['Date'][y]: # uses boolean to check whether dates match
                zing_box.append(zing['Wills Volatility Measure'][y])
                date_box.append(hurricanes['Date'][x])
                hurricane_box.append(hurricanes['Saffir-Simpson Category'][x])
                MA21_box.append(zing['MA21'][y])
                Trailing21_box.append(zing['Trailing MA'][y])
    
    result = pd.DataFrame({'Hurricane Hit Date': date_box,'Hurricane Category': hurricane_box,'Wills Volatility Measure': zing_box, 'Volatility Measure MA': MA21_box, 'Volatility Measure Trailing MA': Trailing21_box})
    return(result)

In [ ]:
a = Align_my_data(ftse)

In [ ]:
a